In [39]:
!pip install kaggle

In [40]:
!mkdir ~/.kaggle

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [41]:
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

In [42]:
! kaggle datasets download anujms/car-damage-detection

car-damage-detection.zip: Skipping, found more recently modified local copy (use --force to force download)


In [43]:
! unzip car-damage-detection.zip

Archive:  car-damage-detection.zip
replace data1a/training/00-damage/0001.JPEG? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [44]:
#importing librabries
import numpy as np
import pandas as pd
import os
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [45]:
#defining the base, train and validation directory path
base_dir = 'data1a'
train_dir = os.path.join(base_dir, 'training')
validation_dir = os.path.join(base_dir, 'validation')

In [46]:
#defining the damage and whole , train nand validation directory
train_damage_dir = os.path.join(train_dir, '00-damage')
train_whole_dir = os.path.join(train_dir, '01-whole')
validation_damage_dir = os.path.join(validation_dir, '00-damage')
validation_whole_dir = os.path.join(validation_dir, '01-whole')

Data Augmentation is used to:

(a) Rescale the data
(b) to bring all the image to same dimension 

In [47]:
#data augmentation
train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

In [48]:
train_generator = train_datagen.flow_from_directory(
        train_dir,  
        target_size=(150, 150), 
        batch_size=16,
        class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
        validation_dir,
        target_size=(150, 150),
        batch_size=16,
        class_mode='binary')

Found 1840 images belonging to 2 classes.
Found 460 images belonging to 2 classes.


In [49]:
model = keras.Sequential()

# Convolutional layer and maxpool layer 1
model.add(keras.layers.Conv2D(32,(3,3),activation='relu',input_shape=(150,150,3)))
model.add(keras.layers.MaxPool2D(2,2))

# Convolutional layer and maxpool layer 2
model.add(keras.layers.Conv2D(64,(3,3),activation='relu'))
model.add(keras.layers.MaxPool2D(2,2))

# Convolutional layer and maxpool layer 3
model.add(keras.layers.Conv2D(128,(3,3),activation='relu'))
model.add(keras.layers.MaxPool2D(2,2))

# Convolutional layer and maxpool layer 4
model.add(keras.layers.Conv2D(128,(3,3),activation='relu'))
model.add(keras.layers.MaxPool2D(2,2))

# This layer flattens the resulting image array to 1D array
model.add(keras.layers.Flatten())

# Hidden layer with 512 neurons and Rectified Linear Unit activation function 
model.add(keras.layers.Dense(512,activation='relu'))

# Output layer with single neuron which gives 0 for Cat or 1 for Dog 
#Here we use sigmoid activation function which makes our model output to lie between 0 and 1
model.add(keras.layers.Dense(1,activation='sigmoid'))

In [50]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [51]:
#steps_per_epoch = train_imagesize/batch_size
#fit data 
model.fit_generator(train_generator,
         steps_per_epoch = 115,
         epochs = 16,
         validation_data = validation_generator
       )

Epoch 1/16


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  """


115/115 [==============================] - 17s 133ms/step - loss: 0.6672 - accuracy: 0.5924 - val_loss: 0.6247 - val_accuracy: 0.6804
Epoch 2/16
115/115 [==============================] - 12s 106ms/step - loss: 0.5960 - accuracy: 0.6957 - val_loss: 0.5539 - val_accuracy: 0.7196
Epoch 3/16
115/115 [==============================] - 12s 107ms/step - loss: 0.5518 - accuracy: 0.7092 - val_loss: 0.5309 - val_accuracy: 0.7543
Epoch 4/16
115/115 [==============================] - 12s 106ms/step - loss: 0.5044 - accuracy: 0.7620 - val_loss: 0.5382 - val_accuracy: 0.7435
Epoch 5/16
115/115 [==============================] - 12s 106ms/step - loss: 0.4404 - accuracy: 0.8016 - val_loss: 0.4781 - val_accuracy: 0.7739
Epoch 6/16
115/115 [==============================] - 13s 109ms/step - loss: 0.3996 - accuracy: 0.8234 - val_loss: 0.4888 - val_accuracy: 0.7543
Epoch 7/16
115/115 [==============================] - 12s 107ms/step - loss: 0.3294 - accuracy: 0.8538 - val_loss: 0.5267 - val_accuracy: 0.7

In [52]:
#weights saving
model.save("classifier.h5")